In [0]:
import pandas as pd
import numpy as np
from numpy import dot
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
import matplotlib.pyplot as plt

import json
import random
import re
import os
import sys
import math
import csv
import pickle
import string

import nltk
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from collections import Counter
from string import digits

import pickle

In [0]:
  !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
  !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
  !apt-get update -qq 2>&1 > /dev/null
  !apt-get -y install -qq google-drive-ocamlfuse fuse
  from google.colab import auth
  auth.authenticate_user()
  from oauth2client.client import GoogleCredentials
  creds = GoogleCredentials.get_application_default()
  import getpass
  !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
  vcode = getpass.getpass()
  !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

  !mkdir -p drive
  !google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
import os

path = "/content/drive/colab notebooks/DM"
os.chdir(path)
os.listdir(path)

['wiki-pages',
 'wikipages',
 'DM.ipynb',
 'q6_input_y_dev',
 'q6_input_x_dev',
 'q6_input_x_train',
 'q6_input_y_train',
 'q6_vocab',
 'q6_glove_model',
 'q6_embedding_matrix',
 'q6.ipynb']

<font face="微软雅黑" size=6 color=#000000> 1. Import data

In [0]:
#load claims
def load_dataset_json(path, instance_num=1e6):
    """
    Reads the Fever Training set, returns list of examples.
    instance_num: how many examples to load. Useful for debugging.
    """
    data = []
    with open(path, 'r') as openfile:
        for iline, line in enumerate(openfile.readlines()):
            data.append(json.loads(line))
            if iline+1 >= instance_num:
                break
    return data

###train
path_windows = "N:\\DesktopSettings\\Desktop\\DM_working\\dataset\\train.jsonl"
path_mac = "/Users/cengqiqi/Desktop/DM_working/dataset/train.jsonl"
dataset = load_dataset_json(path=path_mac)

claims = []
for i in dataset:
    if i['verifiable'] == 'VERIFIABLE':
        claims.append(i)

        
###dev
path_windows = "N:\\DesktopSettings\\Desktop\\DM_working\\dataset\\dev.jsonl"
path_mac = "/Users/cengqiqi/Desktop/DM_working/dataset/dev.jsonl"
dataset = load_dataset_json(path=path_mac)

claims_dev = []
for i in dataset:
    if i['verifiable'] == 'VERIFIABLE':
        claims_dev.append(i)

In [0]:
# load docs lines ()
# format: doc_lines['Alphabetical_list_of_comunes_of_Italy'][1]

def read_lines(wikipedia_dir):

    # doc_id_text saves the title and content of each wiki-page
    doc_id_text=dict()

    for i in tqdm(range(1,110)):# jsonl file number from 001 to 109
        jnum="{:03d}".format(i)
        fname=wikipedia_dir+"wiki-"+jnum+".jsonl"
        with open(fname) as f:
            # point=f.tell()# file pointer starting from 0
            line=f.readline()
            while line:
                data=json.loads(line.rstrip("\n"))
                doc_id=data["id"] 
                text = data["text"]
                lines=data["lines"]
                if text != "":
                    doclines = {}
                    for l in lines.split("\n"):
                        fields = l.split("\t")
                        if fields[0].isnumeric():
                            l_id = int(fields[0])
                            l_txt = fields[1]
                            doclines[l_id] = l_txt

                    doc_id_text[doc_id]=doclines
                # point=f.tell()
                line=f.readline()

    return doc_id_text

wikipedia_dir = '/Users/cengqiqi/Desktop/DM_working/dataset/wiki-pages/wiki-pages/'

doc_lines = read_lines(wikipedia_dir)

In [0]:
def getXandY(claims):
    
    claim_list = []
    sentence_list = []
    y_list = []
    for i in tqdm(range(len(claims))):
        for cla_1 in claims[i]['evidence']:
            for cla_2 in cla_1:
                title = str(cla_2[2])
                sentence = cla_2[3]

                try:
                    sentence_list.append(doc_lines[title][sentence])    
                except:
                    pass
                else:
                    claim_list.append(claims[i]['claim'])
                    if claims[i]['label'] == 'SUPPORTS':
                        y_list.append(1)
                    else:
                        y_list.append(0)
    return [claim_list, sentence_list, y_list]

claim_list, sentence_list, y_list = getXandY(claims)
claim_dev_list, sentence_dev_list, y_dev_list= getXandY(claims_dev)

<font face="微软雅黑" size=6 color=#000000> 2. Prepare embedding models and Input

In [0]:
# import glove model 
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model
glove_model = loadGloveModel("/Users/cengqiqi/Desktop/DM_working/glove/glove.6B.300d.txt")


### this glove_model will be updated in sumPooling() method below

In [0]:
# process the sentence (to a word list before embedding)
def sentenceProcess(sentence):
    #######  lower case  ############################################################
    sentence = sentence.lower()
    
    ####### tokenize ###############################################################
    pattern = r"""(?x)                  
                          (?:[A-Z]\.)+           # abbreviations, e.g. U.S.A. 
                          |\$?\d+(?:,\d+)*(?:\.\d+)?%? # 2,000 or 2.5
                          |\w+(?:[-']\w+)*      # words w/ optional internal hyphens/apostrophe  e.g. can't
                        """
    word_list = nltk.regexp_tokenize(sentence, pattern)
    
    
    ####### remove rrb #############################################################
#     for word in word_list:
#         if word in ['rrb', 'lrb','lsb','rsb']:
#             word_list.remove(word)
    
    ### do not remove stopwords in this case
    ####### remove stop words #######################################################
#     stopwordlist = set(stopwords.words('english'))
#     word_list = [w for w in word_list if w not in stopwordlist]
   
    ####### lemmatize ################################################################
    word_list = [WordNetLemmatizer().lemmatize(w,pos = 'v') for w in word_list] # only Lancaster
    
    return word_list 

In [0]:
#train glove model and build vocab

def trainGloveAndVocab():
    temp_word_list = []

    for i in tqdm(range(len(claim_list))):
        for word in sentenceProcess(claim_list[i]):
            if word not in glove_model:
                # if word not in glove model, update glove model
                embedding_model = Word2Vec([[word]], size=300, window=5, min_count=1, workers=2)
                additional_word = {w: vec for w, vec in zip(embedding_model.wv.index2word, embedding_model.wv.vectors)}
                glove_model.update(additional_word)
            temp_word_list.append(word)

        for word in sentenceProcess(sentence_list[i]):
            if word not in glove_model:
                # if word not in glove model, update glove model
                embedding_model = Word2Vec([[word]], size=300, window=5, min_count=1, workers=2)
                additional_word = {w: vec for w, vec in zip(embedding_model.wv.index2word, embedding_model.wv.vectors)}
                glove_model.update(additional_word)
            temp_word_list.append(word)

    # vectorizer = CountVectorizer()
    # X = vectorizer.fit_transform(temp_word_list)
    # vocab=vectorizer.get_feature_names()

    vocab = list(np.unique(temp_word_list))
    
    return [glove_model, vocab]

glove_model, vocab = trainGloveAndVocab()

In [0]:
with open('q6_glove_model', 'wb') as f:
    pickle.dump(glove_model, f)
with open('q6_vocab', 'wb') as f:
    pickle.dump(vocab, f)

In [0]:
embedding_matrix = np.zeros((len(vocab) + 1, 300))
for i in range(len(vocab)):
    word = vocab[i]
    embedding_vector = glove_model.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
with open("q6_embedding_matrix", "wb") as fp: 
    pickle.dump(embedding_matrix, f)

In [0]:
def getInputX(claim_list, sentence_list):

    input_x1_train = []
    input_x2_train = []

    for i in tqdm(range(len(claim_list))):
        c = []
        for word in sentenceProcess(claim_list[i]):
            try:
                c.append(vocab.index(word))
            except ValueError:
                pass
#                 vocab.append(word)
#                 c.append(len(vocab)-1)
        input_x1_train.append(c)   

        s = []
        for word in sentenceProcess(sentence_list[i]):
            try:
                s.append(vocab.index(word))
            except ValueError:
                pass
#                 vocab.append(word)
#                 s.append(len(vocab)-1)
        input_x2_train.append(s)   

    return [input_x1_train, input_x2_train]

input_x1_train, input_x2_train = getInputX(claim_list, sentence_list)
input_x1_dev, input_x2_dev =  getInputX(claim_dev_list, sentence_dev_list)

In [0]:
print(len(input_x1_train))
print(len(input_x2_train))
print(len(y_list))
print(len(input_x1_dev))
print(len(input_x2_dev))
print(len(y_dev_list))

#save the results
# with open('q6_input_x1_train_no_padding', 'wb') as f:
#     pickle.dump(input_x1_train, f)

# with open('q6_input_x2_train_no_padding', 'wb') as f:
#     pickle.dump(input_x2_train, f)
    
# with open('q6_input_x1_dev_no_padding', 'wb') as f:
#     pickle.dump(input_x1_dev, f)
    
# with open('q6_input_x2_dev_no_padding', 'wb') as f:
#     pickle.dump(input_x2_dev, f)


261975
261975
261975
28341
28341
28341


In [0]:
# with open("q6_input_x1_train_no_padding", "rb") as fp:   # Unpickling
#     input_x1_train = pickle.load(fp)
    
# with open("q6_input_x2_train_no_padding", "rb") as fp:   # Unpickling
#     input_x2_train = pickle.load(fp)
    
# with open("q6_input_x1_dev_no_padding", "rb") as fp:   # Unpickling
#     input_x1_dev = pickle.load(fp)
    
# with open("q6_input_x2_dev_no_padding", "rb") as fp:   # Unpickling
#     input_x2_dev = pickle.load(fp)

In [0]:
from keras.preprocessing.sequence import pad_sequences

# padding
input_x1_train = pad_sequences(input_x1_train,padding='post')
input_x2_train = pad_sequences(input_x2_train,padding='post')

input_x1_dev = pad_sequences(input_x1_dev,maxlen = input_x1_train.shape[1],padding='post')
input_x2_dev = pad_sequences(input_x2_dev, maxlen = input_x2_train.shape[1],padding='post')

input_x_train = np.concatenate((input_x1_train, input_x2_train), axis=1)
input_x_dev = np.concatenate((input_x1_dev, input_x2_dev), axis=1)

In [0]:
input_y_train = np.asarray(y_list)
input_y_dev = np.asarray(y_dev_list)

In [0]:
with open('q6_input_x_train', 'wb') as f:
    pickle.dump(input_x_train, f)
    
with open('q6_input_x_dev', 'wb') as f:
    pickle.dump(input_x_dev, f)

with open('q6_input_y_train', 'wb') as f:
    pickle.dump(input_y_train, f)
    
with open('q6_input_y_dev', 'wb') as f:
    pickle.dump(input_y_dev, f)

<font face="微软雅黑" size=6 color=#000000> 3. Neural Network model

In [0]:
pip install keras-metrics

In [0]:
# import nerual network pakage
from keras.models import Sequential
from keras.layers import Embedding, Input, Lambda, Dense,Flatten, LSTM, Bidirectional,Activation,Dropout, Conv1D, MaxPooling1D,Flatten, regularizers
from keras.models import Model
import keras_metrics
import keras

Using TensorFlow backend.


In [0]:
# load input
with open("q6_input_x_train", "rb") as fp:
    input_x_train = pickle.load(fp)
with open("q6_input_x_dev", "rb") as fp:  
    input_x_dev = pickle.load(fp)
with open("q6_input_y_train", "rb") as fp:  
    input_y_train = pickle.load(fp)
with open("q6_input_y_dev", "rb") as fp:   
    input_y_dev = pickle.load(fp)

input_y_train = np.reshape(input_y_train, (-1,1))
input_y_dev = np.reshape(input_y_dev, (-1,1))

print(input_x_train.shape)
print(input_x_dev.shape)
print(input_y_train.shape)
print(input_y_dev.shape)


with open("q6_glove_model", "rb") as fp:
    glove_model = pickle.load(fp)    
with open("q6_vocab", "rb") as fp:   
    vocab = pickle.load(fp)
with open("q6_embedding_matrix", "rb") as fp:   
    embedding_matrix = pickle.load(fp)

(261975, 224)
(28341, 224)
(261975, 1)
(28341, 1)


In [0]:
embedding_layer = Embedding(len(vocab)+1, 300, weights = [embedding_matrix], input_length = 224, trainable = True, mask_zero = True)
sequence_input = Input(shape=(224,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

x = Bidirectional(LSTM(50,dropout_W=0.2, dropout_U=0.2, return_sequences = True))(embedded_sequences)
x = Bidirectional(LSTM(50,dropout_W=0.2, dropout_U=0.2, return_sequences = True))(embedded_sequences)
x = Bidirectional(LSTM(50,dropout_W=0.2, dropout_U=0.2))(embedded_sequences)
x = Dense(1,activation = 'sigmoid', kernel_regularizer=regularizers.l2(0.01))(x)

model = Model(sequence_input, x)
model.compile(loss='binary_crossentropy',
              optimizer='AdaGrad',
              metrics=['acc', keras_metrics.precision(), keras_metrics.recall()])

print(model.summary())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(50, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(50, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(50, dropout=0.2, recurrent_dropout=0.2)`
  import sys


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 224)               0         
_________________________________________________________________
embedding_18 (Embedding)     (None, 224, 300)          13714500  
_________________________________________________________________
bidirectional_40 (Bidirectio (None, 100)               140400    
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 101       
Total params: 13,855,001
Trainable params: 13,855,001
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
history = model.fit(input_x_train, input_y_train, validation_data=(input_x_dev, input_y_dev), nb_epoch=1000, batch_size=1000, callbacks = [reduce_lr, es])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 261975 samples, validate on 28341 samples
Epoch 1/1000
261975/261975 [==============================] - 464s 2ms/step - loss: 0.3625 - acc: 0.8559 - precision: 0.8580 - recall: 0.9631 - val_loss: 0.5764 - val_acc: 0.7458 - val_precision: 0.6811 - val_recall: 0.9445
Epoch 2/1000
261975/261975 [==============================] - 460s 2ms/step - loss: 0.2857 - acc: 0.8906 - precision: 0.8957 - recall: 0.9631 - val_loss: 0.5593 - val_acc: 0.7547 - val_precision: 0.6945 - val_recall: 0.9276
Epoch 3/1000
261975/261975 [==============================] - 459s 2ms/step - loss: 0.2643 - acc: 0.8997 - precision: 0.9053 - recall: 0.9644 - val_loss: 0.5714 - val_acc: 0.7560 - val_precision: 0.6981 - val_recall: 0.9201
Epoch 4/1000
261975/261975 [==============================] - 458s 2ms/step - loss: 0.2500 - acc: 0.9061 - precision: 0.9118 - recall: 0.9656 - val_loss: 0.5774 - val_acc: 0.7554 - val_precision: 0.6971 - val_recall: 0.9211
Epoch 5/1000
261975/261975 [=========================

In [0]:
# evaluate the model
loss, accr,precision,recall = model.evaluate(input_x_dev,input_y_dev)

28341/28341 [==============================] - 429s 15ms/step
